# Setting up a meaningful DF

In [ ]:
import numpy as np
import pandas as pd
import json
import codecs
import pandas as pd
import matplotlib as m
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize

#Yenny's dups file
json_file = "/Users/kimia/Desktop/Work5/duplicates_batch_2018-05-18.json"
df = pd.read_json(json_file, lines=True)
dfsub = json_normalize(df.submission)

#everything but payload ('twas messy, so I split it then merge)
metadf = dfsub[[ 'task_id', 'updated_at', 'user_id', 'created_at', 'id']]


In [ ]:
dfsub = pd.DataFrame(dict(df.submission[0:])).T
dfpayloads = pd.DataFrame()

# I know this isn't efficient. But it helps split out the payloads (multiple per task_id). 
for i in range(dfsub.shape[0]):
    f = pd.DataFrame(dfsub.payload[i]).T
    f['id'] = dfsub.id[i]
    f['task_id'] = dfsub.task_id[i]
    f['user_id'] = dfsub.user_id[i]
    f['created_at'] = dfsub.created_at[i]
    f['updated_at'] = dfsub.updated_at[i]
    dfpayloads = pd.concat([dfpayloads, f])
    
dfpayloads = dfpayloads.reset_index()
dfpayloads = dfpayloads.rename(index=str, columns={"index": "payload"})
dfpayloads['count'] = 1 # this is to signify one input per user
#dfpayloads.to_csv('dfpayloads.csv')

dfpayloads

In [ ]:
dfpayloads.head()

# Summary Stats

In [254]:
# Percent of time there is not consensus on a single payload within a task

# Group by task, and payload. Annotations will indicate whether consensus or not
df = ((dfpayloads.groupby([ 'task_id', 'payload','annotation']).size()).to_frame()).reset_index()
df['count'] = df[0]

# ID consensus, discrepancies
discrepancies = df[df['count']!=5] 
discrepancies = ((discrepancies.groupby([ 'task_id', 'payload']).count())).reset_index()
        # ^ this is because payloads can be duplicated (so I can't do unique)
consensus = df[df['count']==5] #29918
consensus= consensus.groupby([ 'task_id', 'payload' ]).sum()['count'].reset_index()

# total
total = df.groupby([ 'task_id', 'payload' ]).sum()['count'].reset_index()
         # ^ this is because payloads can be duplicated (so I can't do unique)

consensus1 = 1-(discrepancies.payload.shape[0] /(total.payload.shape[0]))
consensus2 = 1-(discrepancies.payload.shape[0] /(consensus.payload.shape[0] + discrepancies.payload.shape[0] ))

print((consensus1*100),'% of the time there is consensus on a single payload within a task.',
      'Assuming the consensus is generally correct, this suggests Taskus is an effective resource')



89.05574923274038 % of the time there is consensus on a single payload within a task. Assuming the consensus is generally correct, this suggests Taskus is an effective resource


In [315]:
## For each user, how often do How often is this person the only person who disagrees?
lowagreement = dfpayloads.groupby(['payload', 'task_id', 'annotation']).size().to_frame(name = 'count').reset_index()
lowagreement = lowagreement[lowagreement['count'] == 1]
result = pd.merge(lowagreement,dfpayloads,  how = 'right', on=['payload', 'task_id','annotation'])

differ_from_others = ((result.groupby('user_id').count()['count_x'].to_frame())).sort_values('count_x', ascending=False).reset_index()

#total number of annotations that each person does 
total = dfpayloads.groupby(['user_id']).count()['payload'].to_frame(name = 'count').reset_index()

#calculations
perfmstat = pd.merge(differ_from_others,total,  on=['user_id'])
perfmstat['%'] = (perfmstat['count_x']/perfmstat['count'])*100
perfmstat = perfmstat.rename(index=str, columns={"count_x": "#timediffer", 'count': 'totalannot'})
print("Top users who are the *only* Taskus employee diverging from consensus (5)")
perfmstat.sort_values('%', ascending=False).reset_index()



Top users who are the *only* Taskus employee diverging from consensus (5)


,index,user_id,#timediffer,totalannot,%
0,0,30,270,5997,4.502251
1,2,22,166,4771,3.479354
2,1,29,192,7571,2.535993
3,3,37,152,7008,2.168950
4,6,38,105,5967,1.759678
5,4,46,120,7671,1.564333
6,5,21,116,7823,1.482807
7,8,36,80,6031,1.326480
8,12,40,71,6027,1.178032
9,7,39,83,7582,1.094698


In [339]:

xx = (dfpayloads[dfpayloads.user_id==30]).groupby([ 'annotation']).size().to_frame(name = 'count').reset_index()


print(29/(xx['count'].sum()))
xx


0.004835751208937802


,annotation,count
0,duplicate,550
1,non-duplicate,5418
2,unsure,29


In [340]:
xx = dfpayloads
xx = xx.groupby([ 'annotation']).size().to_frame(name = 'count').reset_index()
xx

,annotation,count
0,duplicate,9773
1,non-duplicate,158229
2,unsure,533


In [341]:
533/(xx['count'].sum())

0.0031625478387278605

In [346]:
lowagreement = dfpayloads.groupby(['payload', 'task_id', 'annotation']).size().to_frame(name = 'count').reset_index()
lowagreement = lowagreement[lowagreement['count'] == 1]
result = pd.merge(lowagreement,dfpayloads,  how = 'right', on=['payload', 'task_id','annotation'])
r30results = (result[result.user_id == 30]).groupby(['annotation']).size().to_frame(name = 'count').reset_index()
r30results

,annotation,count
0,duplicate,550
1,non-duplicate,5418
2,unsure,29


In [367]:
r = result.groupby(['annotation']).size().to_frame(name = 'count').reset_index()
533/(r['count']).sum()

0.0031625478387278605